In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Look-through Expansion by Conserved Quantity

Attributes
----------
valuation
transactions
instruments
recipes
securitised portfolios
lookthrough
exposure
"""

toggle_code("Hide docstring")

The following notebook details how to perform lookthrough expansion of indices by different quantities, such as exposure, i.e. exposure is conserved by expansion, and is apportioned out to lookthrough constituents according to specified weights in a lookthrough portfolio. This allows, for example, a reference portfolio with 50% weights on each of two constituents to apportion 50% of its PV to each constituent, or 50% of its exposure, as configured in the valuation recipe.

In the case of equity indices, expansion by exposure is identical to expansion by PV, since PV = exposure for equities. However, in the event that an index contains a derivative such as an equity option, the choice of conserved quantity can lead to different results, as shown in this notebook. These results represent different 'views' or breakdowns of the index by its constituents.

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
import uuid
from datetime import datetime
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from flatten_json import flatten

import os
import pandas as pd
import math

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

#Load LUSID API Components
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
reference_portfolio_api = api_factory.build(lusid.api.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

# Set Global Scope
global_scope = "LookthroughByConservedQuantity"
pricing_date = "2022-01-07T00:00:00Z"
option_maturity_date = "2023-01-07T00:00:00Z"

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.11013.0


In [3]:
# MSFT and call option on NVDA inside a equally-weighted reference portfolio
# Load publicly listed equities
def load_equity(ticker, client_internal_id):
    client_internal = "Instrument/default/ClientInternal"
    
    equity = models.Equity(
        instrument_type="Equity",
        dom_ccy="USD",
    )

    equity_definition = models.InstrumentDefinition(
        name=ticker,
        identifiers={"ClientInternal": models.InstrumentIdValue(client_internal_id),
                     "RIC": models.InstrumentIdValue(ticker)
                    },
        definition=equity,
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
    luid = upsert_response.values[client_internal].lusid_instrument_id
    print(f"LUID for newly created equity with ticker {ticker}: {luid}")
    return luid

MSFT_luid = load_equity("MSFT", "eq_us_MSFT")
NVDA_luid = load_equity("NVDA", "eq_us_NVDA")

print ("Equities Upserted!")

option = models.EquityOption(instrument_type="EquityOption",
    start_date=pricing_date,
    option_maturity_date=option_maturity_date,
    option_settlement_date=option_maturity_date,
    delivery_type = "Cash",
    option_type= "Call",
    strike=300.0,
    dom_ccy="USD",
    underlying_identifier="RIC",
    code="NVDA",
   )

option_definition = models.InstrumentDefinition(
    name="NVDA Call Option",
    identifiers={"ClientInternal": models.InstrumentIdValue("opt_us_NVDA")},
    definition=option)

upsert_request = {"NVDA_option": option_definition}
upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
option_luid = upsert_response.values["NVDA_option"].lusid_instrument_id

print ("Options Upserted!")

LUID for newly created equity with ticker MSFT: LUID_00004B8L
LUID for newly created equity with ticker NVDA: LUID_00004B8M
Equities Upserted!
Options Upserted!


In [4]:
# Create a Reference Portfolio for our index
def load_ref_portfolio(portfolio_code):
    try:
        response = reference_portfolio_api.create_reference_portfolio(
            scope=global_scope,
            create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
                display_name=portfolio_code,
                base_currency="USD",
                code=portfolio_code, 
                created="2022-01-01",
                instrument_scopes=[global_scope]
            ),
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

load_ref_portfolio("MSFT_and_NVDA-option")

# Initialise a list to hold our constituents
constituents = [
        models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/LusidInstrumentId": luid
        },
        weight=0.5,
        currency="USD",
        ) for luid in [MSFT_luid, option_luid]]

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from="2022-01-07T00:00:00Z",
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = api_factory.build(
    lusid.api.ReferencePortfolioApi
).upsert_reference_portfolio_constituents(
     scope=global_scope,
     code="MSFT_and_NVDA-option",
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted!")


Could not create a portfolio with id 'MSFT_and_NVDA-option' because it already exists in scope 'LookthroughByConservedQuantity'.
Constituents Upserted!


In [5]:
index = models.SimpleInstrument(
    instrument_type="SimpleInstrument",
    dom_ccy="USD",
    asset_class="Equities",
    simple_instrument_type="Index"
)

index_definition = models.InstrumentDefinition(
    name="index_inst",
    identifiers={"ClientInternal": models.InstrumentIdValue("index_inst")},
    definition=index,
    look_through_portfolio_id=models.ResourceId(scope=global_scope, code="MSFT_and_NVDA-option")
)

upsert_request = {"index_inst": index_definition}
upsert_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    scope=global_scope,
    request_body=upsert_request
)

lookthroughinst_luid = upsert_response.values["index_inst"].lusid_instrument_id
print(f"LUID for newly created securitised instrument: {lookthroughinst_luid}")

LUID for newly created securitised instrument: LUID_00004B8O


In [6]:
# Create our Transaction Portfolios
def load_txn_portfolio(portfolio_code):
    try:
        transaction_portfolio_api.create_portfolio(
            scope=global_scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency="USD",
                created="2022-01-01",
                instrument_scopes=[global_scope]
            ),
        )
        print("Portfolio: " + portfolio_code + " loaded!")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

load_txn_portfolio("MyPortfolio")

upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=global_scope,
        code="MyPortfolio",
        transaction_request=[
            models.TransactionRequest(
                transaction_id="Transaction1",
                type="Buy",
                instrument_identifiers={"Instrument/default/LusidInstrumentId": lookthroughinst_luid},
                transaction_date=pricing_date,
                settlement_date=pricing_date,
                units=100,
                transaction_price=models.TransactionPrice(
                    price=0, type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=0, currency="USD"
                ),
            )
        ],
    )

print ("Transactions upserted!")

Could not create a portfolio with id 'MyPortfolio' because it already exists in scope 'LookthroughByConservedQuantity'.
Transactions upserted!


In [7]:
# Upsert equity quotes into LUSID
def upsert_quote(ticker, ticker_type, price):
    instrument_quotes = {
    ticker: models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=ticker,
                    instrument_id_type=ticker_type,
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=pricing_date,
            ),
            metric_value=models.MetricValue(value=price, unit="USD"),        
        )
    }
    
    response = quotes_api.upsert_quotes(scope=global_scope, request_body=instrument_quotes) 

upsert_quote("MSFT", "RIC", 329)
upsert_quote("NVDA", "RIC", 293)
upsert_quote("index_inst", "ClientInternal", 1000)

print("Quotes upserted!")

Quotes upserted!


In [8]:
# Upsert market data for Black-Scholes evaluation of our option

def upsert_complex_market_data(complex_market_data, asset_name):
    complex_id = models.ComplexMarketDataId(provider="Lusid",
                                            effective_at=pricing_date,
                                            market_asset=asset_name)

    upsert_request = models.UpsertComplexMarketDataRequest(market_data_id=complex_id,
                                                              market_data=complex_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertComplexMarketData
    response = complex_market_data_api.upsert_complex_market_data(
        scope=global_scope,
        request_body={"1": upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload complex market data {response.failed}")

    print(f"Complex market data {asset_name} uploaded into scope={global_scope}")
    

def upsert_ois_yield_curve(ccy):
    # provide the structured data file source and it's document format
    complex_market_data = models.DiscountFactorCurveData(
        base_date = pricing_date,
        dates = [pricing_date],
        discount_factors= [1.0],
        market_data_type="DiscountFactorCurveData"
    )
    upsert_complex_market_data(complex_market_data, ccy + "/" + ccy + "OIS")
    
def upsert_flat_vol_surface(option, vol):
    complex_market_data = models.EquityVolSurfaceData(market_data_type="EquityVolSurfaceData",
                                                      base_date=pricing_date,
                                                      instruments=[option],
                                                      quotes=[models.MarketQuote("LogNormalVol", vol)]
                                                     )
    upsert_complex_market_data(complex_market_data, "NVDA/USD/LN")

upsert_ois_yield_curve("USD")
upsert_flat_vol_surface(option, 0.05)


Complex market data USD/USDOIS uploaded into scope=LookthroughByConservedQuantity
Complex market data NVDA/USD/LN uploaded into scope=LookthroughByConservedQuantity


Whether to perform lookthrough expansion, and if so, which quantity to conserve, is determined by the recipe. It requires instructing the recipe via model rules to tell instruments of particular types to expand by the 'Sum' (or, alternatively, 'AbsoluteSum') scaling methodology, then specifying a conserved quantity in the pricing options. 

There is a third type of expansion, the 'Unity' methodology, that simply replaces each unit of an instrument with one 'unit' of the portfolio that it refers to. In this case, no quantities are conserved, and the choice of conserved quantity is ignored.

In [9]:
def UpsertRecipe(recipe_code, conserved_quantity):
    if conserved_quantity is not None:
        expansion_options = models.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions")
    else:
        expansion_options = None
        
    model_rules=[]
    model_rules.append(models.VendorModelRule(
            supplier="Lusid",
            model_name="SimpleStatic",
            instrument_type="SimpleInstrument",
            model_options=expansion_options
    ))
    model_rules.append(models.VendorModelRule(
        supplier="Lusid",
        model_name="BlackScholes",
        instrument_type="EquityOption",
        model_options=expansion_options
    ))
    
    configuration_recipe = models.ConfigurationRecipe(
        scope=global_scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.RIC.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                ),
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                )
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=global_scope,
                attempt_to_infer_missing_fx=True             
            ),
        ),
        pricing=models.PricingContext(
            model_rules=model_rules,
            options=models.PricingOptions(
                conserved_quantity_for_lookthrough_expansion=conserved_quantity)
        ),    
    )

    upsert_configuration_recipe_response = (
        configuration_recipe_api.upsert_configuration_recipe(
            upsert_recipe_request=models.UpsertRecipeRequest(
                configuration_recipe=configuration_recipe
            )
        )
    )
    
UpsertRecipe("DontExpandRecipe", None)
UpsertRecipe("ExpandByPvRecipe", "PV")
UpsertRecipe("ExpandByExposureRecipe", "Exposure")

print("Recipes upserted!")

Recipes upserted!


In [10]:
# Create a valuation function allowing users to specify the look-through recipe
def get_daily_val(date, portfolio_code, recipe_code, group_by = None):
    group_by = [group_by] if group_by else []
    metricsList = []
    columnsToRename={
            f"Instrument/default/Name": "Instrument Name",
            f"Valuation/PvInReportCcy": "PV (Reporting Ccy)",
            f"Valuation/ExposureInReportCcy": "Exposure (Reporting Ccy)",
            f"Holding/default/FundLineage": "Fund Lineage",
            f"Holding/default/Units": "Implied Holding Units"
        }
    
    metricsList.extend([
            models.AggregateSpec(f"Instrument/default/Name", "Value"),
            models.AggregateSpec(f"Holding/default/FundLineage", "Value"),
            models.AggregateSpec(f"Holding/default/Units", "Value"),
            models.AggregateSpec(f"Valuation/PvInReportCcy", "Value"),        
            models.AggregateSpec(f"Valuation/ExposureInReportCcy", "Value"),
            models.AggregateSpec(f"Lookthrough/Apportioned/Valuation/PvInReportCcy", "Value"),
            models.AggregateSpec(f"Lookthrough/Apportioned/Valuation/ExposureInReportCcy", "Value"),
        ])
          
    # Build and run valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=global_scope, code=recipe_code),
        metrics=metricsList,
        group_by=group_by,
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=global_scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
        report_currency = "USD"
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data
    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    return vals_df

In [11]:
get_daily_val(pricing_date, "MyPortfolio", "DontExpandRecipe")

,Instrument Name,Fund Lineage,Implied Holding Units,PV (Reporting Ccy),Exposure (Reporting Ccy),Lookthrough/Apportioned/Valuation/PvInReportCcy,Lookthrough/Apportioned/Valuation/ExposureInReportCcy
0,index_inst,MyPortfolio,100.00,"100,000.00","100,000.00",None,None


In [12]:
# Expansion by PV apportions half of the index's PV to each constituent (PV is the 'conserved value')
# This implies a number of holding units of each constituent, that can be used to compute other analytics
# Note that as a result, the sum of exposures of the constituents doesn't equal the exposure of the index
# To see a rough breakdown of the index's exposure by constituents, use the Lookthrough Apportionment keys
get_daily_val(pricing_date, "MyPortfolio", "ExpandByPvRecipe")

,Instrument Name,Fund Lineage,Implied Holding Units,PV (Reporting Ccy),Exposure (Reporting Ccy),Lookthrough/Apportioned/Valuation/PvInReportCcy,Lookthrough/Apportioned/Valuation/ExposureInReportCcy
0,MSFT,MyPortfolio/MSFT_and_NVDA-option,151.98,"50,000.00","50,000.00","50,000.00","50,000.00"
1,NVDA Call Option,MyPortfolio/MSFT_and_NVDA-option,"16,335.39","50,000.00","45,909.10","50,000.00","50,000.00"


In [13]:
# Meanwhile, expansion by exposure natively apportions half of the index's exposure to each constituent
# In this case, the sum of PVs of the constituents doesn't equal to PV of the index
get_daily_val(pricing_date, "MyPortfolio", "ExpandByExposureRecipe")

,Instrument Name,Fund Lineage,Implied Holding Units,PV (Reporting Ccy),Exposure (Reporting Ccy),Lookthrough/Apportioned/Valuation/PvInReportCcy,Lookthrough/Apportioned/Valuation/ExposureInReportCcy
0,MSFT,MyPortfolio/MSFT_and_NVDA-option,151.98,"50,000.00","50,000.00","50,000.00","50,000.00"
1,NVDA Call Option,MyPortfolio/MSFT_and_NVDA-option,"17,791.02","54,455.44","50,000.00","50,000.00","50,000.00"
